<a href="https://colab.research.google.com/github/won-seop/CAPSTONE-DESIGN/blob/main/ViewsPredictionModel(bilstm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions
!pip install keras
!pip install konlpy
!pip install scikeras

In [ ]:
import torch
torch.cuda.is_available()

True

전처리

In [ ]:
import pandas as pd
import re
import contractions
import spacy
import en_core_web_sm
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Embedding, Dense, Dropout, Flatten, LSTM, Bidirectional
import matplotlib.pyplot as plt
import numpy as np
import keras
from konlpy.tag import Okt
from konlpy.tag import Kkma
from keras.models import load_model
import math
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#LOAD DATA
nt = pd.read_csv('/content/drive/MyDrive/Capstone Design/youtubeData/totalYoutubeData_3.csv', encoding='utf-8')
nt = nt[['title_data', 'views_data']]
nt.columns = ['title', 'views']
print(nt.dtypes)


#문자열이 아닌 행 제거
nt = nt[nt['title'].apply(lambda x: isinstance(x, str))]
print(nt['title'].apply(type).unique())

#특수문자 제거
nt['title'] = nt['title'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', x))

# 'title' 컬럼에서 NaN 값이 있는 행 제거
nt = nt.dropna(subset=['title'])
nt = nt.dropna(subset=['views'])

#공백으로 이루어진 행을 제거
nt = nt[nt['title'].str.strip() != '']

print(nt)
# IQR 계산
Q1 = np.percentile(nt['views'], 25)
Q3 = np.percentile(nt['views'], 75)
IQR = Q3 - Q1

# 이상치의 상한값과 하한값 계산
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# 이상치를 식별하여 제거
filtered_data = nt[(nt['views'] >= lower_bound) & (nt['views'] <= upper_bound)]

# 이상치가 제거된 데이터셋
df=filtered_data['views']
print(filtered_data)
print(filtered_data['views'].mean())
top_10_views = filtered_data['views'].nlargest(10)

title     object
views    float64
dtype: object
[<class 'str'>]
                                                   title    views
0                                         나홀로 아이폰 사러 오사카   6409.0
1      김광석님이라니 대한민국 과학기술대전의 놀라운 현장 걱정하지 마세요 미래는 준비돼 있습니다   1153.0
2                         노션 사용기 업무용 단기계획용 이력서용 노션 만드는 법   3992.0
3               갤럭시 어떻게 나올까 정말 네이밍이 바뀔까 들려오는 소식들 정리했습니다   23508.0
4         갤럭시버즈라이브 에어팟프로 비교 들어갈게 최신 무선이어폰 사운드 통화연결 노캔 비교  24715.0
...                                                  ...      ...
89573                                    거대한 갭메우기가 시작됩니다  10959.0
89574  통계 사상 경기 인천 최대 폭등 기록 예전 가격으로 다시 돌아가지 않는 인플레이션과...  25273.0
89575  화폐개혁 그냥 떠본게 아닌 것 같다 디플레이션 공포와 부동산 시장의 미래는 놀부부동...  83201.0
89576  속보 만기 주택담보대출 중단 나이 제한 걸리고 있네요 서두르셔야 할 것 같습니다 놀...   2600.0
89577          전월세 신고제 시작 임차 시장 자금 흐름 투명화와 그 결과는 바로 이것이다  47154.0

[89577 rows x 2 columns]
                                                   title    views
0                                         나홀로 아이폰 사러

In [ ]:
# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# '조회수' 열 데이터 추출
views_data = filtered_data['views'].values.reshape(-1, 1)  # 2차원 배열로 변환

# 조회수 데이터 정규화 (0에서 1 사이의 값으로 스케일링)
views_normalized = scaler.fit_transform(views_data)

# 정규화된 값을 'normalized_data' 열에 추가
filtered_data['normalized_data'] = views_normalized

<ipython-input-111-157fe1bcfb9f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['normalized_data'] = views_normalized


In [ ]:
nomalized_title=filtered_data['title']
nomalized_views=filtered_data['normalized_data']
nomalized_data={'title':nomalized_title,'nomalized_views':nomalized_views}
nomalized_df=pd.DataFrame(nomalized_data)
nomalized_df

,title,nomalized_views
0,나홀로 아이폰 사러 오사카,0.026714
1,김광석님이라니 대한민국 과학기술대전의 놀라운 현장 걱정하지 마세요 미래는 준비돼 있습니다,0.004806
2,노션 사용기 업무용 단기계획용 이력서용 노션 만드는 법,0.016640
3,갤럭시 어떻게 나올까 정말 네이밍이 바뀔까 들려오는 소식들 정리했습니다,0.097987
4,갤럭시버즈라이브 에어팟프로 비교 들어갈게 최신 무선이어폰 사운드 통화연결 노캔 비교,0.103018
...,...,...
89573,거대한 갭메우기가 시작됩니다,0.045680
89574,통계 사상 경기 인천 최대 폭등 기록 예전 가격으로 다시 돌아가지 않는 인플레이션과...,0.105344
89575,화폐개혁 그냥 떠본게 아닌 것 같다 디플레이션 공포와 부동산 시장의 미래는 놀부부동...,0.346802
89576,속보 만기 주택담보대출 중단 나이 제한 걸리고 있네요 서두르셔야 할 것 같습니다 놀...,0.010837


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(filtered_data['title'], filtered_data['normalized_data'], test_size=0.2, random_state=42)

텍스트 토큰화

In [ ]:
okt = Okt()
X_train_tokenized = X_train.apply(lambda x: okt.morphs(x))

텍스트 벡터화

In [ ]:
# Tokenizer를 사용하여 단어를 숫자 시퀀스로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_tokenized)
X_train_sequences = tokenizer.texts_to_sequences(X_train_tokenized)

# 시퀀스 데이터를 패딩하여 길이를 맞춤
max_length = max([len(seq) for seq in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# 임베딩 매트릭스 생성
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, idx in tokenizer.word_index.items():
    try:
        embedding_vector = word2vec_model.wv[word]
        # 'gta'와 같은 단어가 없을 경우 embedding_vector는 None이 됩니다.
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    except KeyError:
        # Word2Vec 모델에 해당 단어가 없는 경우 KeyError가 발생하며 embedding_vector는 None이 됩니다.
        embedding_vector = None

In [ ]:
'''
print(word2vec_model.wv.vectors.shape)
wv_result=word2vec_model.wv.most_similar("데이트")
print(wv_result)
'''

모델 학습

In [ ]:

#모델 정의
model = Sequential()

# 임베딩 레이어 설정
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1,
                            output_dim=100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)  # 사전 훈련된 임베딩을 사용하므로 trainable을 False로 설정

model.add(embedding_layer)
model.add(Bidirectional(LSTM(32)))  # 양방향 LSTM 레이어
model.add(Dense(1, activation='linear'))  # 회귀 모델을 위한 출력 레이어

# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# 모델 요약 출력
model.summary()

# 모델 학습
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
X_test_tokenized = X_test.apply(lambda x: okt.morphs(x))

# LSTM 모델에 입력할 형태로 시퀀스 데이터 변환
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokenized)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

In [ ]:
# LSTM 모델에서 예측값 계산
predictions = model.predict(X_test_padded)

# 정규화된 조회수 열 데이터로 예측값 역변환 (원래 스케일로 변환)
predicted_views = scaler.inverse_transform(predictions)

y_test_np = np.array(y_test).reshape(-1, 1)
y_test_original_scale = scaler.inverse_transform(y_test_np)

# 예측값과 실제값 간의 평가 (예를 들어, 평균 제곱 오차 계산)
rmse = mean_squared_error(y_test_original_scale, predicted_views)**0.5
print(f"Root Mean Squared Error: {rmse}")

In [ ]:
'''
predict_data={'title':X_test,'predicted_views':predicted_views,'y_test':y_test_original_scale_list}
predict_df=pd.DataFrame(predict_data)
predict_df
'''

In [ ]:
'''
# BiLSTM 모델을 생성하는 함수 정의
def create_bilstm_model(units=64):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                        output_dim=100,
                        input_length=max_length,
                        weights=[embedding_matrix],  # 임베딩 매트릭스 설정
                        trainable=False))  # 임베딩 레이어의 가중치 동결
    model.add(Bidirectional(LSTM(units=units, return_sequences=True)))
    model.add(Bidirectional(LSTM(units=units)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# KerasRegressor로 모델 래핑
model = KerasRegressor(build_fn=create_bilstm_model, units=64, epochs=10, batch_size=64, verbose=1)

# 하이퍼파라미터 탐색을 위한 그리드 설정
param_dist = {
    'units': [32, 64, 128],  # BiLSTM 레이어의 유닛 수에 대한 후보 값들
}

# 랜덤 서치를 이용한 하이퍼파라미터 탐색
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train_padded, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters: ", random_search.best_params_)

# 최적의 모델 얻기
best_model = random_search.best_estimator_
'''

In [ ]:
'''
X_test_tokenized = X_test.apply(lambda x: okt.morphs(x))

# LSTM 모델에 입력할 형태로 시퀀스 데이터 변환
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokenized)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

predictions = model.predict(X_test_padded)

# 정규화된 조회수 열 데이터로 예측값 역변환 (원래 스케일로 변환)
predicted_views = scaler.inverse_transform(predictions)

y_test_np = np.array(y_test).reshape(-1, 1)
y_test_original_scale = scaler.inverse_transform(y_test_np)

# 예측값과 실제값 간의 평가 (예를 들어, 평균 제곱 오차 계산)
mse = mean_squared_error(y_test_original_scale, predicted_views)
mae = mean_absolute_error(y_test_original_scale, predicted_views)
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
'''

In [ ]:
'''
# PLOT
plt.plot(best_model.history['loss'])
plt.plot(best_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
'''

모델 저장

In [ ]:
model.save("/content/drive/MyDrive/Capstone Design/youtubeViewsPrediction/BiLstm.h5")

In [ ]:
#PREDICT NEW TITLE
with open('/content/drive/MyDrive/Capstone Design/youtubeViewsPrediction/lstm.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

모델 테스트

In [ ]:
from keras.models import load_model
import pickle
with open('/content/drive/MyDrive/Capstone Design/youtubeViewsPrediction/lstm.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
model = load_model('/content/drive/MyDrive/Capstone Design/youtubeViewsPrediction/BiLstm.h5')

def predictView(title):
    import math

    input_str = title
    input_df = pd.DataFrame(columns=['title', 'view'])
    input_df = input_df.append({'title' : input_str} , ignore_index=True)
    seq = loaded_tokenizer.texts_to_sequences(input_df['title'].values)
    padded = pad_sequences(seq, 58)
    pred = math.exp(model.predict(padded))
    return pred

#TEST

print(predictView("다이어트시 자주하는 3가지 실수"))